```1.``` Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция – удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.


https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f

In [1]:
from collections import Counter, defaultdict
from nltk import sent_tokenize
import gzip
import csv
import pandas as pd
import re
from operator import itemgetter
import nltk

In [2]:
data = pd.read_csv('lenta-ru-news.csv')

In [3]:
texts = data['text']

In [4]:
def align_words(sent_1, sent_2):
    tokens_1, tokens_2 = normalize(sent_1), normalize(sent_2)
    return zip(tokens_1, tokens_2)

In [5]:
def normalize(text):
    pat = re.compile(r'[А-Яа-я]+')
    normalized = re.findall(pat, text)

    return [word.lower() for word in normalized if word]

In [6]:
corpus = []
for text in texts[:12000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [7]:
vocab = set()

for sent in corpus:
    vocab.update(sent)

In [8]:
words = Counter()

for sent in corpus:
    words.update(sent)

In [9]:
word_freq = dict(words)

for key, value in word_freq.items():
    word_freq[key] = value/len(words)

In [10]:
def n_deletes_helper(word, min_len=0):
    result = set()
    # taking word length into account
    if len(word) > min_len:
        for i in range(len(word)):
            result.add(word[:i] + word[i+1:])
    else:
        result.add(word)
    return result

In [12]:
def n_deletes(word, edit_distance=2, min_len=2):
    edit_distance -= 1
    forms = list(n_deletes_helper(word, min_len))
    tmp = set(forms)
    while edit_distance > 0:
        edit_distance -= 1
        new_forms = []
        for form in tmp:
            new_forms.extend(n_deletes_helper(form, min_len))
        tmp = set(new_forms)
        forms.extend(tmp)
        
    return forms

In [13]:
n_deletes('word', edit_distance=2)

['ord', 'wrd', 'wor', 'wod', 'wd', 'wo', 'wr', 'od', 'rd', 'or']

In [14]:
sym_vocab = {}
for word in vocab:
    sym_vocab[word] = word
    forms = n_deletes(word)
    for form in forms:
        if form not in sym_vocab.keys():
            sym_vocab[form] = word
        else:
            if isinstance(sym_vocab[form], set):
                sym_vocab[form].add(word)
            else:
                sym_vocab[form] = {sym_vocab[form]}
                sym_vocab[form].add(word)

In [15]:
for word in sym_vocab['односоронни']:
    print(word_freq[word], word)

3.077514906712829e-05 односторонние
1.5387574533564147e-05 односторонним
7.693787266782073e-06 односторонний
3.077514906712829e-05 односторонних


In [16]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
good = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [17]:
def most_probable(words, word_freq=word_freq):
    freqs = dict()
    for word in words:
        freqs[word] = word_freq[word]
    return max(freqs.items(), key=itemgetter(1))[0]

In [18]:
def vocab_helper(word, vocab=vocab):
    return word in vocab

In [19]:
'вообще' in vocab

True

In [20]:
sym_vocab['вбще'], sym_vocab['вобще']

('вообще', 'вообще')

In [21]:
def correct(word, sym_vocab=sym_vocab):
    if vocab_helper(word):
        return word
    # we don't expect to reliably spellcheck two- or one-letter words
    elif len(word) < 3:
        return word
    else:
        if word in sym_vocab.keys(): 
            candidates = sym_vocab[word]
            if isinstance(candidates, set):
                return most_probable(candidates)
            else:
                return candidates
        else:
            forms = n_deletes(word)
            candidates = []
            for form in forms:
                if form in sym_vocab.keys():
                    cand = sym_vocab[form]
                    if isinstance(cand, set):
                        candidates.extend(cand)
                    else:
                        candidates.append(cand)
            if candidates:
                return most_probable(candidates)
            else:
                # the word is expected to have errors in it,
                # but we don't have a form to replace it with
                return f'*{word}'

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

In [129]:
def metrics(corrections, keep_good=False, g_list=None):
    result = defaultdict(lambda: 0)
    for (good, bad, corrected) in corrections:
        #процент правильных слов
        if good == corrected:
            result['good_corrections'] += 1

        # процент исправленных ошибок
        if (good != bad):
            result['original_errors'] += 1
            if (good == corrected):
                result['errors_fixed'] += 1
                if keep_good:
                    g_list.append((good, bad, corrected))
            
            
        # процент ошибочно исправленных правильных слов
        if (good == bad):
            result['original good'] += 1
            if (good != corrected):
                result['bad_corrections'] += 1
            
    # процент правильных слов
    result['good_corrections'] /= len(corrections)
    # процент исправленных ошибок
    result['errors_fixed'] /= result['original_errors']
    # процент ошибочно исправленных правильных слов
    result['bad_corrections'] /= result['original good']
    
    print(f"Процент правильных слов: {result['good_corrections'] * 100}")
    print(f"Процент исправленных ошибок: {result['errors_fixed'] * 100}")
    print(f"процент ошибочно исправленных правильных слов {result['bad_corrections'] * 100}")

In [130]:
cache = dict()
corrections = []

for i in range(len(good)):
    pairs = align_words(good[i], bad[i])
    for pair in pairs:
        left = pair[0]
        right = pair[1]
        if right in cache.keys():
            correction = left, right, cache[right]
        else:
            corrected_word = correct(right)
            correction = left, right, corrected_word
            cache[right] = corrected_word
        corrections.append(correction)

    

In [131]:
g_list = []
metrics(corrections, keep_good=True, g_list=g_list)

Процент правильных слов: 80.2791625124626
Процент исправленных ошибок: 30.36041539401344
процент ошибочно исправленных правильных слов 9.984510901942095


В целом работает не так уж плохо, хотя слова делить всё равно не умеет.

In [135]:
Counter(g_list).most_common(10)

[(('сегодня', 'седня', 'сегодня'), 24),
 (('вообще', 'вобще', 'вообще'), 19),
 (('естественно', 'естесственно', 'естественно'), 17),
 (('хочется', 'хочеться', 'хочется'), 16),
 (('очень', 'ооочень', 'очень'), 15),
 (('ничего', 'ничо', 'ничего'), 9),
 (('как', 'както', 'как'), 8),
 (('что', 'чтото', 'что'), 7),
 (('периодически', 'переодически', 'периодически'), 7),
 (('получается', 'получаеться', 'получается'), 6)]

```2.``` Добавьте к полученному алгоритму исправления триграммную модель и проверьте, улучшает ли она качество.

In [136]:
tri_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in corpus:
    for w1, w2, w3 in nltk.trigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        tri_model[(w1, w2)][w3] += 1

In [137]:
for bigram in tri_model:
    total_count = sum(tri_model[bigram].values())
    for target in tri_model[bigram]:
        tri_model[bigram][target] /= total_count

In [138]:
good_grams = [['<s>', '<s>'] + normalize(sent) + ['</s>', '</s>'] for sent in good]
bad_grams = [['<s>', '<s>'] + normalize(sent) + ['</s>', '</s>'] for sent in bad]

In [139]:
def most_probable_grams(candidates, preceding, model=tri_model, word_freq=word_freq):
    model = model[preceding]
    possibilities = dict()
    for candidate in candidates:
        if candidate in model.keys():
            #possibilities.append((candidate, model[candidate]))
            possibilities[candidate] = model[candidate]

    if not possibilities:
        possibilities = dict()
        for candidate in candidates:
            possibilities[candidate] = word_freq[candidate]
            
    return max(possibilities.items(), key=itemgetter(1))[0]
    

In [140]:
most_probable_grams(sym_vocab['не'], preceding=('<s>', '<s>'))

'не'

In [141]:
def correct_grams(word, preceding, sym_vocab=sym_vocab):
    if vocab_helper(word):
        return word
    # having context allows checking all words
    #elif len(word) < 3:
    #    return word
    else:
        if word in sym_vocab.keys(): 
            candidates = sym_vocab[word]
            if isinstance(candidates, set):
                return most_probable_grams(candidates, preceding)
            else:
                return candidates
        else:
            forms = n_deletes(word)
            candidates = []
            for form in forms:
                if form in sym_vocab.keys():
                    cand = sym_vocab[form]
                    if isinstance(cand, set):
                        candidates.extend(cand)
                    else:
                        candidates.append(cand)
            if candidates:
                return most_probable(candidates)
            else:
                # the word is expected to have errors in it,
                # but we don't have a form to replace it with
                return f'*{word}' 

In [142]:
cache = dict()
corrections = []

for i in range(len(good_grams)):

    append = False
    pairs = zip(good_grams[i], bad_grams[i])
    trail = []
    for pair in pairs:
        
        if pair[1] == '<s>':
            trail.append(pair[1])
        
        elif pair[1] == '</s>':
            continue
        
        else:
            trail.append(pair[1])

            trail = trail[-3:]
            preceding = tuple(trail[:2])

            left = pair[0]
            right = pair[1]
            
            if right in cache.keys():
                correction = left, right, cache[right]
            else:
                corrected_word = correct_grams(right, preceding)
                correction = left, right, corrected_word
                cache[right] = corrected_word
            corrections.append(correction)

    

In [143]:
g_list = []
metrics(corrections, keep_good=True, g_list=g_list)

Процент правильных слов: 80.24310052804623
Процент исправленных ошибок: 30.474452554744524
процент ошибочно исправленных правильных слов 10.008340283569641


In [145]:
Counter(g_list).most_common(10)

[(('сегодня', 'седня', 'сегодня'), 24),
 (('вообще', 'вобще', 'вообще'), 19),
 (('естественно', 'естесственно', 'естественно'), 17),
 (('хочется', 'хочеться', 'хочется'), 16),
 (('очень', 'ооочень', 'очень'), 15),
 (('ничего', 'ничо', 'ничего'), 9),
 (('как', 'както', 'как'), 8),
 (('что', 'чтото', 'что'), 7),
 (('периодически', 'переодически', 'периодически'), 7),
 (('получается', 'получаеться', 'получается'), 6)]

Попробуем склеивать размноженные буквы

In [146]:
def is_multed(word):
    if re.search(r'(.)\1\1', word):
        return True
    else:
        return False

In [147]:
def deduplicate(word):
    letters = re.findall(r'(.)\1\1', word)
    for letter in letters:
        word = re.sub(f'{letter}' + r'{3,}', f'{letter}', word)
    return word

In [148]:
def correct_grams_fancy(word, preceding, sym_vocab=sym_vocab):
    if is_dupl(word):
        word = deduplicate(word)
    if vocab_helper(word):
        return word
    else:
        if word in sym_vocab.keys(): 
            candidates = sym_vocab[word]
            if isinstance(candidates, set):
                return most_probable_grams(candidates, preceding)
            else:
                return candidates
        else:
            forms = n_deletes(word)
            candidates = []
            for form in forms:
                if form in sym_vocab.keys():
                    cand = sym_vocab[form]
                    if isinstance(cand, set):
                        candidates.extend(cand)
                    else:
                        candidates.append(cand)
            if candidates:
                return most_probable(candidates)
            else:
                # the word is expected to have errors in it,
                # but we don't have a form to replace it with
                return f'*{word}' 

In [149]:
cache = dict()
corrections = []

for i in range(len(good_grams)):

    append = False
    pairs = zip(good_grams[i], bad_grams[i])
    trail = []
    for pair in pairs:
        
        if pair[1] == '<s>':
            trail.append(pair[1])
        
        elif pair[1] == '</s>':
            continue
        
        else:
            trail.append(pair[1])

            trail = trail[-3:]
            preceding = tuple(trail[:2])

            left = pair[0]
            right = pair[1]
            
            if right in cache.keys():
                correction = left, right, cache[right]
            else:
                corrected_word = correct_grams_fancy(right, preceding)
                correction = left, right, corrected_word
                cache[right] = corrected_word
            corrections.append(correction)

Наблюдаем некоторый прирост в качестве

In [150]:
g_list = []
metrics(corrections, keep_good=True, g_list=g_list)

Процент правильных слов: 80.39254757397629
Процент исправленных ошибок: 31.386861313868614
процент ошибочно исправленных правильных слов 10.008340283569641


In [151]:
Counter(g_list).most_common(10)

[(('сегодня', 'седня', 'сегодня'), 24),
 (('вообще', 'вобще', 'вообще'), 19),
 (('естественно', 'естесственно', 'естественно'), 17),
 (('хочется', 'хочеться', 'хочется'), 16),
 (('очень', 'ооочень', 'очень'), 15),
 (('очень', 'оооочень', 'очень'), 9),
 (('ничего', 'ничо', 'ничего'), 9),
 (('как', 'както', 'как'), 8),
 (('что', 'чтото', 'что'), 7),
 (('периодически', 'переодически', 'периодически'), 7)]